# PEARC 25 Hands-on Exercises

Login to https://training.tapis.io. Use the username and password provided to you to authenticate.
![Login Step](images/Login-user-password.png)

## Systems

In this section we create two Tapis systems, one for running on a VM host using FORK and one for running on an HPC type host using BATCH.

Note that although it is possible, we have not provided any login credentials in the system definitions.
Well-crafted system definitions are likely to be copied and re-used, so, for security reasons, it is recommended that
login credentials be registered using separately.

When you login to TapisUI you may see My Systems as 0, since we did not create any systems yet. As we create systems, the list will get populated.


### Create System
Click on Systems from the Left tab, click on Create System, right most button. You should see two ways to create system.
By entering information in the form or using a json editor. 
We will be using JSON editor. Copy the below system json in the json editor and click Create System.
![Login Step](images/CreateSystem.png)

In [ ]:
# change the id to reflect your training user number, rest should remain unchanged.
{
  "id": "pearc25-vm-training_user1",
  "description": "Test system",
  "systemType": "LINUX",
  "host": "129.114.35.138",
  "effectiveUserId":"${apiUserId}",
  "defaultAuthnMethod": "PASSWORD",
  "rootDir": "/",
  "canExec": true,
  "jobRuntimes": [ { "runtimeType": "DOCKER" } ],
  "jobWorkingDir": "HOST_EVAL($HOME)/sharetest/workdir"
}

In the My Systems you should now see the system you just created.

### Register Credentials for the VM system

After creating the system, you will need to register credentials for your username. These will be used by Tapis to
access the host. Various authentication methods can be used to access a system, such as PASSWORD and PKI_KEYS. We will should you how to add password credentials for your system. You will need to use **vm_password** provided to you.
![Add Credential](images/AddCredentialsSystem.png)


Click on the AUTHENTICATE and add Password
![Add Password](images/addpassword.png)

You should see Successfully created credentials. If there is any error please try to enter the vm_password again
![Credential Success](images/credentialsSuccess.png)

Now that we have sucessfully added credentials, our System is ready for use.
Try to list files from the Files tab in the left bar. You should see something similar to the image below.
![Files List](images/FilesList.png)

We will now learn to create a Sentiment Analysis Application with Tapis.

### Natural Language Processsing: Sentiment Analysis
- Sentiment Analysis is one of the most popular applications of Natural Language Processing, which uses the Text Classification method to analyse the sentiment or emotion of the given text.
- Sentiment analysis assigns a label like 🙂 positive, 🙁 negative, or 😐 neutral to a sequence of text.
- It is useful tool to make business decisions based on customer feedback and reviews.



In [ ]:
#!pip install -q transformers

In [ ]:
from transformers import pipeline

In [ ]:
sentiment_pipeline = pipeline("sentiment-analysis")

In [ ]:
text= "Glad to see you at PEARC 25"
sentiment_pipeline(text)

### Create App with Tapis UI

In the Apps Tab in TapisUI, click on + NEW APP > JSON EDITOR and paste the below json 

In [ ]:
{
    "id": "pearc25-sentiment-analysis",
    "version": "0.1",
    "description": "Application utilizing the sentiment analysis model from Hugging Face.",
    "jobType": "FORK",
    "runtime": "DOCKER",
    "containerImage": "tapis/sentiment-analysis:2.0.3",
    "jobAttributes": {
        "parameterSet": {
            "archiveFilter": {
                "includeLaunchFiles": false
            }
        },
        "memoryMB": 1,
        "nodeCount": 1,
        "coresPerNode": 1,
        "maxMinutes": 10
    }
}

Click on Create App and it should give a success message.

### Submit Job in Tapis UI

In the Apps Tab, click on the Application you created. You should see Submit job button, click that and enter the below job json in the JSON EDITOR, and click submit.

![Submit Job](images/submitjobjson.png)

In [ ]:
{
    "name":"sentiment analysis",
    "description":"sentiment analysis with hugging face transformer pipelines",
    "appId":"pearc25-sentiment-analysis-training1",
    "appVersion":"0.1",
    "execSystemId":"pearc25-vm-training1", 
    "parameterSet": {
    "appArgs": [
            {"arg": "--sentences"},
            {"arg": "\"This is great\" \"This is not fun\""},
            {"arg": "--output-filepath"},
            {"arg":"/TapisOutput/results.csv"}
            
        ]
    }
}

Once the job successfully finishes you should see a results.csv file which has the results from sentiment analysis on the sentences provided in the job
![Job Success](images/jobcomplete.png)

### Create a system for the HPC cluster

With just a few changes to the system definition you can create a second system that can be used to run the
same application on an HPC type host. Note the minimal changes:

* **id** - A unique id is required
* **host** - Main hostname for the HPC system.
* **rootDir** - Using the root directory of the host gives us flexibility in setting **jobWorkingDir**.
  Note that you still need LINUX permissions.
* **jobWorkingDir** - Now determined dynamically using the Tapis v3 function HOST_EVAL()
* **jobRuntimes** - Most HPC systems support singularity and not docker
* **batchLogicalQueue.hpcQueueName** - HPC queue to use by default.
* **batchLogicalQueues** - HPC queue definitions for this HPC system.

In the Tapis UI, Systems Tab. Click on + CREATE SYSTEM, copy the below json in JSON EDITOR and click CREATE SYSTEM.

In [ ]:
{
  "id": "pearc25_system_id_hpc",
  "description": "System for testing jobs on an HPC type host for Gateways24",
  "systemType": "LINUX",
  "host": "129.114.35.138",
  "defaultAuthnMethod": "PASSWORD",
  "effectiveUserId": "${apiUserId}",
  "rootDir": "/",
  "canExec": true,
  "jobRuntimes": [ { "runtimeType": "SINGULARITY" } ],
  "jobWorkingDir": "HOST_EVAL($HOME)/sharetest/workdir",
  "canRunBatch": true,
  "batchScheduler": "SLURM",
  "batchSchedulerProfile": "tacc",
  "batchDefaultLogicalQueue": "tapisNormal",
  "batchLogicalQueues": [
    {
      "name": "tapisNormal",
      "hpcQueueName": "normal",
      "maxJobs": 50,
      "maxJobsPerUser": 10,
      "minNodeCount": 1,
      "maxNodeCount": 16,
      "minCoresPerNode": 1,
      "maxCoresPerNode": 68,
      "minMemoryMB": 1,
      "maxMemoryMB": 16384,
      "minMinutes": 1,
      "maxMinutes": 60
    }
  ]
}

Once the HPC System is successfully created it should show up in My Systems.

### Register Credentials for the HPC system

As before, now you will need to register credentials for your username. Use the vm_password to add credentials.

Once you get a message that system credentials were successfully created, you should be able to list files on that system.
Go to the files tab and click on the system-id that was created recently.
![Files List HPC](images/files-list-hpc.png)

## HPC Application

In order to run a job on a system you will need to create a Tapis application.

### Create an application that can be run on the VM host or the HPC cluster

In [ ]:
app_id_hpc = "gateways24-sentiment-analysis-hpc-" + username
app_def_hpc= {
    "id": app_id_hpc,
    "version": "0.1",
    "description": "Application utilizing the sentiment analysis model from Hugging Face.",
    "jobType": "BATCH",
    "runtime": "SINGULARITY",
    "runtimeOptions": ["SINGULARITY_RUN"],
    "containerImage": "/tmp/sentiment-analysis_1.0.1.sif",
    "jobAttributes": {
            "parameterSet": {
            "archiveFilter": {
                "includeLaunchFiles": False
            }
        },
        "memoryMB": 1,
        "nodeCount": 1,
        "coresPerNode": 1,
        "maxMinutes": 10
    }
}

In [ ]:
client.apps.createAppVersion(**app_def_hpc)

#client.apps.patchApp(**app_def_hpc, appId=app_id_hpc, appVersion='0.1')

In [ ]:
# List all applications available to you
print("****************************************************")
print("List all applications")
print("****************************************************")
client.apps.getApps()

In [ ]:
# Get details for the application you created
print("****************************************************")
print("Fetch application: " + 'Sentiment Analysis HPC app')
print("****************************************************")
client.apps.getAppLatestVersion(appId=app_id_hpc)

In [ ]:
# Submit job to run the sentiment analysis application
pa= {
    "parameterSet": {
    "appArgs": [
            {"arg": "--sentences"},
            {"arg": "\"This is great\" \"This is not fun\""}
            
        ]
    }}

# Submit a job
job_response_hpc=client.jobs.submitJob(name='sentiment analysis',description='sentiment analysis with hugging face transformer pipelines',appId=app_id_hpc,appVersion='0.1',execSystemId=system_id_hpc, **pa)


### Get Job submission response


In [ ]:
# Get Job submission response
print("****************************************************")
print("Job Submitted: " + app_id_hpc)
print("****************************************************")
print(job_response_hpc)

### Get Job UUID from the submission response


In [ ]:
# Get job uuid from the job submission response
print("****************************************************")
job_uuid_hpc=job_response_hpc.uuid
print("Job UUID: " + job_uuid_hpc)
print("****************************************************")

### Check the status of the job


In [ ]:
# Check the status of the job
print("****************************************************")
print(client.jobs.getJobStatus(jobUuid=job_uuid_hpc))
print("****************************************************")

### Download output of the job


In [ ]:
# Once the job is in the FINISHED state, you can download output of the job
print("Job Output file:")

print("****************************************************")
jobs_output_hpc= client.jobs.getJobOutputDownload(jobUuid=job_uuid_hpc,outputPath='results.csv')
print(jobs_output_hpc)
print("****************************************************")

### Setting Notifications on Job events


Note: Make sure to add your email address in the submitJob call.

In [ ]:
pa= {
    "parameterSet": {
    "appArgs": [
            {"arg": "--sentences"},
            {"arg": "\"This is great\" \"This is not fun\""}
            
        ]
    }}

# Submit a job
job_response_hpc_email=client.jobs.submitJob(name='sentiment analysis',description='sentiment analysis with hugging face transformer pipelines',appId=app_id_hpc,appVersion='0.1',execSystemId=system_id_hpc,subscriptions= [ { "description": "Test subscriptions", "eventCategoryFilter": "ALL","deliveryTargets": [ { "deliveryMethod": "EMAIL","deliveryAddress":"<Enter your email>"}] }],**pa)


In [ ]:
# Get Job submission response
print("****************************************************")
print("Job Submitted: " + app_id_hpc)
print("****************************************************")
print(job_response_hpc_email)

In [ ]:
# Get job uuid from the job submission response
print("****************************************************")
job_uuid_hpc_email=job_response_hpc_email.uuid
print("Job UUID: " + job_uuid_hpc_email)
print("****************************************************")

In [ ]:
# Check the status of the job
print("****************************************************")
print(client.jobs.getJobStatus(jobUuid=job_uuid_hpc_email))
print("****************************************************")

### Cancel a job


In [ ]:
# If necessary, you can cancel a long running job.
# To cancel a running job
# client.jobs.cancelJob(jobUuid=job_uuid_vm)

## Share System and App

In [ ]:
#Making your execution system Public
client.systems.shareSystemPublic(systemId=system_id_hpc)

In [ ]:
# Making the app public
client.apps.shareAppPublic(appId=app_id_hpc)

In [ ]:
# Get Share info on the app
client.apps.getShareInfo(appId=app_id_hpc)
# Now any user in the tenant should be able to run your application

In [ ]:
# Unsharing public app
#client.apps.unShareAppPublic(appId=app_id_hpc)

In [ ]:
## You should now be able to run any public apps
'''
pa= {
    "parameterSet": {
    "appArgs": [
            {"arg": "--sentences"},
            {"arg": "\"This is great\" \"This is not fun\""}
            
        ]
    }}

# Submit a job
job_response_hpc_email=client.jobs.submitJob(name='sentiment analysis',description='sentiment analysis with hugging face transformer pipelines',appId=app_id_hpc,appVersion='0.1',execSystemId=system_id_hpc,subscriptions= [ { "description": "Test subscriptions", "eventCategoryFilter": "ALL","deliveryTargets": [ { "deliveryMethod": "EMAIL","deliveryAddress":"<Enter your email>"}] }],**pa)
'''